In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB0
from tqdm import tqdm
from tensorflow_addons.metrics import FBetaScore 

In [ ]:
x_train = []
x_test = []
y_train = []

df_train = pd.read_csv('/content/train_v2.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

print(f'{labels}\n{label_map}\n{inv_label_map}')

In [ ]:
for i, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread(f'/content/train-jpg/{i}.jpg')
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255

print(f'{x_train.shape}\n{y_train.shape}')

In [ ]:
split = 35000
x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

print(f'x_train : {x_train.shape} \ny_train : {y_train.shape}')
print(f'x_valid : {x_valid.shape} \ny_valid : {y_valid.shape}')

In [ ]:
fbeta = FBetaScore(
        num_classes=len(labels),
        average='weighted',
        beta=2.0,
        threshold=0.2,
        name='fbeta')

def model_efficientnet_b0(input_shape=(32, 32, 3), weight_path=None):
    backbone = EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape)

    model = Sequential()
    model.add(BatchNormalization(input_shape=input_shape))
    model.add(backbone)
    model.add(Flatten())
    model.add(Dense(17, activation='sigmoid'))

    model.compile(
        loss='binary_crossentropy', 
        optimizer='adam',
        metrics=[fbeta, tf.keras.metrics.AUC()])
    
    return model

model = model_efficientnet_b0()

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

lr_decay = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=3, 
    verbose=1, 
    mode='auto', 
    epsilon=0.0001, 
    cooldown=0, 
    min_lr=0.0000001)

In [ ]:
model_hist = model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=100,
    verbose=1,
    validation_data=(x_valid, y_valid),callbacks=[es,mc,lr_decay])

In [ ]:
epochs = range(1, len(model_hist.history['loss']) + 1)

plt.figure(figsize = (9, 6))
plt.plot(epochs, model_hist.history['fbeta'])
plt.title('Train History')
plt.xlabel('Epochs')
plt.ylabel('Result')
plt.legend(['F-beta Score'])
plt.grid()
plt.show()